# Edge-Probing Fine-tuning Example

In this notebook, we will:

* Train a RoBERTa base model on Edge-Probing (Semeval) and evaluate its performance
* Because the Edge-Probing data is not publicly available, we will simulate the run with a single example. This will serve as a guide for users who have access to the task data, or similarly formatted data.
* **The encoder is not frozen for training runs in this notebook.**

The code shown in this notebook will work, but the results will not be representative of the task!

## `jiant` Pipeline

In [ ]:
import sys
sys.path.insert(0, "/content/jiant")
import jiant.proj.main.tokenize_and_cache as tokenize_and_cache
import jiant.proj.main.export_model as export_model
import jiant.proj.main.scripts.configurator as configurator
import jiant.proj.main.runscript as main_runscript
import jiant.shared.caching as caching
import jiant.utils.python.io as py_io
import jiant.utils.display as display
import jiant.utils.retokenize as retok
import os
from tqdm import tqdm
from rich.progress import track
import traceback
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging

In [ ]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb

In [ ]:
editor = Editor()

def build_mono_example(sentence, labels, index):
  tokens = sentence.split()
  targets = []
  for i, tok in enumerate(tokens):
    target = {"span": [i, i+1],
              "label": labels[i],
              "info": {"comment": ""}}
    targets.append(target)
  index += 1
  return {"text": sentence,
             "info": {"id": index},
             "targets": targets}

In [ ]:
examples = []
id = -1

In [ ]:
templates = ['Not many {human} love lions as pets',
             'Not many {human} view squirrels as food',
             'Not many {human} want foxes as friends',
             'Not many {human} eat turtle as dinners',
             'Not many {human} use horses as weapons',]

human = ['people','farmers']
sentences = []
for temp in templates:
  sentences += editor.template(temp, human=human).data[0:2]
labels = '+ - = - - - -'.split()
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
templates = ['Many {human} love dogs as pets',
             'Many {human} view chickens as food',
             'Many {human} want cows as friends',
             'Many {human} eat fish as dinners',
             'Many {human} use horses for transportation',]

human = ['people','kids']
sentences = []
for temp in templates:
  sentences += editor.template(temp, human=human).data[0:2]
labels = '+ = + + + +'.split()
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
templates = ['A few {human} chased a girl',
             'A few students learned some {skill}']
human = ['boys', 'policemen', 'teachers', 'trainers', 'mothers']
skill = ['sports', 'instruments', 'poetries', 'experiments', 'languages']
sentences = []
for temp in templates:
  sentences += editor.template(temp, human=human, skill=skill).data[0:5]
labels = '+ + + + + +'.split()
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
templates = ['Not every teenage boy likes {activity}.',
             'Not every graduate student studies {field}.']
activity = ['competitive sports', 'loud concerts', 'drinking parties', 'hard exams', 'long hikings']
field = ['Computational Semantic', 'Software Development', 'Cognitive Neroscience', 'Abstract Algebra', 'advanced engineering']
sentences = []
for temp in templates:
  sentences += editor.template(temp, activity=activity, field=field).data[0:5]
labels = '+ - + + - - -'.split()
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
templates = ['Most {subj} who {verbp} {adv}, do not {verbs} {obj}.',
             'Most activities that require {sup}, are not very {adj}.']
subj = ['dogs', 'students', 'horses', 'athelets', 'managers']
verbp = ['bark', 'sleep', 'run', 'train', 'work']
adv = ['loudly', 'endlessly', 'fast', 'hard', 'smartly']
verbs = ['like', 'attend', 'eat', 'need', 'make']
obj = ['cats',   'classes',  'apples', 'vacations', 'mistakes']
sup = ['energy', 'experience', 'skills', 'stradegies', 'money']
adj = ['relaxing', 'simple', 'ordinary', 'boring', 'fair']
sentences = []

for i in range(5):
  sentences.append(
    templates[0].replace('{subj}', subj[i]).replace('{verbp}', verbp[i]).replace('{adv}', adv[i]).replace('{verbs}', verbs[i]).replace('{obj}', obj[i]))

for i in range(5):
  sentences.append(
    templates[0].replace('{sup}', sup[i]).replace('{adj}', adj[i]))

labels = '+ = = = = - + - -'.split()
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
templates = ['The {subj} who {verbp} {adv}, does not {verbs} any {obj}',
             'The activitie that require {sup}, can not miss any {noun}']
subj = ['dog', 'student', 'horse', 'athelet', 'manager']
verbp = ['barks', 'sleeps', 'runs', 'trains', 'works']
adv = ['loudly', 'endlessly', 'fast', 'hard', 'smartly']
verbs = ['like', 'attend', 'eat', 'need', 'make']
obj = ['cats',   'classes',  'apples', 'vacations', 'mistakes']
sup = ['energy', 'experience', 'skills', 'stradegies', 'money']
noun = ['nutrition', 'dedication', 'knowledge', 'plannings', 'fundraisers']
sentences = []
for i in range(5):
  sentences.append(
    templates[0].replace('{subj}', subj[i]).replace('{verbp}', verbp[i]).replace('{adv}', adv[i]).replace('{verbs}', verbs[i]).replace('{obj}', obj[i]))

for i in range(5):
  sentences.append(
    templates[0].replace('{sup}', sup[i]).replace('{noun}', noun[i]))
labels = '+ = = = = - + - - -'.split()
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]

In [ ]:
sentences = ['Three out of five dentists recommend that their patients brush their teeth at least four times a day',
 'Two out of five coaches recommend that their trainers lift heavy weights at least three times a week',
 'Seven out of ten teachers recommend that their students read their notes at least seven times a week',
 'four out of five scientists recommend that young adults drink their water at least fifteen times a day',
 'five out of nine doctors recommend that their patients rest their mind at least two times a day']
labels = '= - - = + + + + + + + + + + - + + +'.split()
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
labels = '+ + - + + + + +'.split()
templates = ['At least seven {animal} died yesterday in Morocco',
             'At least four fish died yesterday in {country}']

animal = ['dogs', 'cats', 'cows', 'horses', 'squirrels']
country = ['Morocco', 'England', 'Spain', 'Italy', 'Korea']
sentences = []
for temp in templates:
  sentences += editor.template(temp, animal=animal, country=country).data[0:5]
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
labels = '+ + + - - -'.split()
templates = ['At most 23 dogs are {adj}',
             'At most 77 {animal} are hungry']

animal = ['dogs', 'cats', 'cows', 'horses', 'squirrels']
adj = ['nice', 'interesting', 'cute', 'soft', 'strong']
sentences = []
for temp in templates:
  sentences += editor.template(temp, animal=animal, adj=adj).data[0:5]
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:

labels = '+ + - + +'.split()
templates = ['At least 5 people {verb}',
             'At least 9 {animal} ran']

animal = ['dogs', 'cats', 'cows', 'horses', 'squirrels']
verb = ['studied', 'jumped', 'saw', 'walked', 'danced']
sentences = []
for temp in templates:
  sentences += editor.template(temp, animal=animal, verb=verb).data[0:5]
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
labels = '+ + + - -'.split()
templates = ['Less than 5 people {verb}',
             'Less than 9 {animal} ran']

animal = ['dogs', 'cats', 'cows', 'horses', 'squirrels']
verb = ['studied', 'jumped', 'saw', 'walked', 'danced']
sentences = []
for temp in templates:
  sentences += editor.template(temp, animal=animal, verb=verb).data[0:5]
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
labels = '+ + - + + +'.split()
templates = ['More than 15 dogs are {adj}',
             'More than 36 {animal} are hungry']

animal = ['dogs', 'cats', 'cows', 'horses', 'squirrels']
adj = ['nice', 'interesting', 'cute', 'soft', 'strong']
sentences = []
for temp in templates:
  sentences += editor.template(temp, animal=animal, adj=adj).data[0:5]
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
labels = '+ = = = ='.split()
templates = ['Exactly 10 dogs are {adj}',
             'Exactly 5 {animal} are sleeping']

animal = ['dogs', 'cats', 'cows', 'horses', 'squirrels']
adj = ['nice', 'interesting', 'cute', 'soft', 'strong']
sentences = []
for temp in templates:
  sentences += editor.template(temp, animal=animal, adj=adj).data[0:5]
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
labels = '+ + + + + + + + ='.split()
templates = ['Some but not all students cheated in this {obj}',
             'Some but not all students {verb} in this class']

obj = ['class', 'field', 'sports', 'competition', 'club']
verb = ['studied', 'slept', 'failed', 'succeeded', 'learned']
sentences = []
for temp in templates:
  sentences += editor.template(temp, obj=obj, verb=verb).data[0:5]
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
labels = '+ + - + + + + ='.split()
templates = ['More than three students {verb} in this class',
             'More than seven students cheated in this {obj}']

obj = ['class', 'field', 'sports', 'competition', 'club']
verb = ['studied', 'slept', 'failed', 'succeeded', 'learned']
sentences = []
for temp in templates:
  sentences += editor.template(temp, obj=obj, verb=verb).data[0:5]
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
labels = '+ + + - - - - - ='.split()
templates = ['Less than three students {verb} in this class',
             'Less than seven students got awards in the {obj}']

obj = ['class', 'field', 'conference', 'competition', 'club']
verb = ['studied', 'slept', 'failed', 'succeeded', 'learned']
sentences = []
for temp in templates:
  sentences += editor.template(temp, obj=obj, verb=verb).data[0:5]
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
labels = "+ + + - =".split()
templates = ['More {animal} than cats sit',
            'More lions than cows {verb}']

animal = ['dogs', 'cats', 'cows', 'horses', 'squirrels']
verb = ['ate', 'slept', 'drank', 'ran', 'moved']
sentences = []
for temp in templates:
  sentences += editor.template(temp, animal=animal, verb=verb).data[0:5]
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
labels = "+ - + + =".split()
templates = ['Fewer {animal} than cats sit',
            'Fewer lions than cows {verb}']

animal = ['dogs', 'cats', 'cows', 'horses', 'squirrels']
verb = ['ate', 'slept', 'drank', 'ran', 'moved']
sentences = []
for temp in templates:
  sentences += editor.template(temp, animal=animal, verb=verb).data[0:5]
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
labels = "+ - - - - + +".split()
sentences = ["Every student's favorite class was French",
             "Every student's best class was Psychology",
             "Every parents' favorite thing was kids",
             "Every dog's favorite food was meat",
             "Every animals's essential resource was water",
             "Every boy's favorite sport was baseball",
             "Every children's favorite activity was swimming",
             "Every student's dream job was engineering",
             "Every farmer's worst nightmare was storms",
             "Every person's best memory was summer"]
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
labels = "+ - - - - - -".split()
sentences = ["No students liked any class in Chemistry",
             "No customers liked any trash in resturants",
             "No players disliked any running in baseball",
             "No singers liked any fighting in concerts",
             "No coaches liked any lazyness in practice",
             "No farmers hated any cities in Kansas",
             "No doctors watched any movies in cinemas",
             "No programmers missed any bugs in softwares",
             "No visitors touched any paintings in museums",
             "No hunters hunted any aligators in Everglades"]
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
labels = "+ + = = =".split()
templates = ["All but five {animal} died",
            "All but seven dogs {verb}"]

animal = ['dogs', 'cats', 'cows', 'horses', 'squirrels']
verb = ['ate', 'slept', 'drank', 'jumped', 'ran']
sentences = []
for temp in templates:
  sentences += editor.template(temp, animal=animal, verb=verb).data[0:5]
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
labels = "+ - + - + + + +".split()
sentences = ["If every cat runs, some dogs run also",
             "If every teacher compete, some students compete also",
             "If every parent learns, some kids learn also",
             "If every fish swims, some ducks siwm also",
             "If every tree grows, some flowers grow also",
             "If every container opens, some boxes open also",
             "If every phone rings, some speaker ring also",
             "If every car moves, some bikes move also",
             "If every building stands, some bridges stand also",
             "If every players wins, some rookies win also"]
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
labels = "+ - - - + - -".split()
sentences = ["If some dog runs, no cat sits",
             "If some students studies, no exam fails",
             "If some computer crashes, no data stays",
             "If some signals disapears, no call comes",
             "If some tree grows, no animal rests",
             "If some assitants stuck, no experiment completes",
             "If some cells degenerate, no muscle heals",
             "If some phone ring, no battery charges",
             "If some car stop, no gas leaks",
             "If some boats float, no water moves"]
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
labels = "+ + + + -".split()
sentences = ["John is dancing without clothes",
             "John is writing without papers",
             "John is eating without forks",
             "John is sleeping without pillows",
             "John is flying without wings",
             "John is running without shoes",
             "John is swimming without hats",
             "John is singing without lyrics",
             "John is filming without mics",
             "John is jumping without hesitation"]
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
labels = "+ + + - + -".split()
templates = ['Ursula {verb_down} to sing or dance',
            'Ursula refused to {verb} or dance']

verb_down = ['refused', 'declined', 'rejected', 'repudiate', 'rebuffed']
verb = ['eat', 'sleep', 'drink', 'jump', 'run']
sentences = []
for temp in templates:
  sentences += editor.template(temp, verb_down=verb_down, verb=verb).data[0:5]
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
labels = "+ + + = + +".split()

templates = ['A {animal} ate 2 rotten biscuits',
            'A cat took 5 {adj_obj}']

animal = ['dogs', 'cats', 'cows', 'horses', 'squirrels']
adj_obj = ['delicious cake', 'sweet barries', 'red apples', 'small fish', 'yellow balls']
sentences = []
for temp in templates:
  sentences += editor.template(temp, animal=animal, adj_obj=adj_obj).data[0:5]
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
templates = ['Three {animal} love four {animal}',
             'Five {animal} saw Seven {animal}']

animal = ['dogs', 'cats', 'cows', 'horses', 'squirrels']
sentences = []
for temp in templates:
  sentences += editor.template(temp, animal=animal).data[0:5]
labels = '= = + = ='.split()
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
templates = ['My {human_relate} said I could have three candies',
 'My {human_relate} said I could watch two movies',
 'My {human_relate} said they could read four chapters',
 'My {human_relate} said I could sleep five hours',
 'My {human_relate} said they could run six miles']

sentences = []
human_relate = ['parents', 'friends']
for temp in templates:
  sentences += editor.template(temp, human_relate=human_relate).data[0:2]
labels = '+ + + + + + = +'.split()
examples += [build_mono_example(sentence, labels, id) for sentence in sentences]
len(examples)

In [ ]:
with open('data/gold_label_replicate.txt', 'r', encoding='utf-8') as pola1:
  lines = pola1.readlines()
  for line in lines:
    data = line.split(';')
    if len(data) > 1:
      sentence = str(data[0])
      sentence = sentence.replace(u'\xa0', u' ')
      labels = data[1].split()
      examples.append(build_mono_example(sentence, labels, id))
len(examples)

In [ ]:
with open('data/gold_mono2.txt', 'r', encoding='utf-8') as pola2:
  lines = pola2.readlines()
  for line in lines:
    data = line.split(';')
    if len(data) > 1:
      sentence = str(data[0])
      sentence = sentence.replace(u'\xa0', u' ')
      labels = data[1].split()
      examples.append(build_mono_example(sentence, labels, id))
len(examples)

In [ ]:
with open('data/Dev_mono.json', 'r', encoding='utf-8') as pola2:
  lines = pola2.readlines()
  for line in lines:
    data = line.split(';')
    if len(data) > 1:
      sentence = str(data[0])
      sentence = sentence.replace(u'\xa0', u' ')
      labels = data[1].split()
      examples.append(build_mono_example(sentence, labels, id))
len(examples)

In [ ]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

signature_2_label = {
  '+': "yes",
  '-': "no",
  'o': "maybe"
}

def build_example(sentence, verb, signature, index):
  tokens = sentence.split()
  for toki, tok in enumerate(tokens):
    if verb in lemmatizer.lemmatize(tok, 'v'):
      break
  if tokens[toki-1] == "not":
    span1 = [0, toki-2]
  else:
    span1 = [0, toki]
  span2 = [toki+2, len(tokens)]
  span1_str = tokens[span1[0]:span1[1]]
  span2_str = tokens[span2[0]:span2[1]]
  example = {"text": sentence,
             "info": {"id": index},
             "targets": [
               {"label": signature_2_label[signature], 
                "span1": span1, "span2": span2, 
                "info": {"comment": f"{span1_str} => {span2_str}"}}]}
  return example

factuality_val_data = []

for index, row in tqdm(df.iterrows()):
  verb = row['verb']
  sent_pos = row['sentence']
  sent_pos = sent_pos.strip("'").replace(",", "")
  sent_neg = row['neg_sentence']
  sent_neg = sent_neg.strip("'").replace(",", "")
  signatures = row['signature'].split('/')
  
  example_pos = build_example(sent_pos, verb, signatures[0], index)
  factuality_val_data.append(example_pos)
  example_neg = build_example(sent_neg, verb, signatures[1], index)
  factuality_val_data.append(example_neg)

In [ ]:
df_train = pd.read_csv('data/mega-veridicality-v2.csv')
df_train = df_train.drop(['participant','list','nativeenglish','exclude'], axis=1)
df_train = df_train.groupby('frame').apply(lambda x: x.sample(n=2000)).reset_index(drop = True)
df_train.head(3)

In [ ]:
def build_example_train(sentence, verb, label, index):
  tokens = sentence.split()
  for toki, tok in enumerate(tokens):
    if verb in lemmatizer.lemmatize(tok, 'v'):
      break
  if tokens[toki-1] == "not":
    span1 = [0, toki-2]
  else:
    span1 = [0, toki]
  span2 = [toki+2, len(tokens)]
  span1_str = tokens[span1[0]:span1[1]]
  span2_str = tokens[span2[0]:span2[1]]
  example = {"text": sentence,
             "info": {"id": index},
             "targets": [
               {"label": label, 
                "span1": span1, "span2": span2, 
                "info": {"comment": f"{span1_str} => {span2_str}"}}]}
  return example

factuality_train_data = []
exceptions = []

for index, row in tqdm(df_train.iterrows()):
  verb = row['verb']
  sentence = row['sentence']
  sentence = sentence.strip("'")
  label = row['veridicality']
  example = build_example_train(sentence, verb, label, row['presentationorder'])
  factuality_train_data.append(example)

In [ ]:
predictor_ner = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/ner-elmo.2021-02-12.tar.gz")

In [ ]:
transitive_train = py_io.read_json("data/transitive_relation_train.json", mode="r")

In [ ]:
transitive_val = py_io.read_json("data/transitive_relation.json", mode="r")
len(transitive_val)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, _, _ = train_test_split(transitive_train, [0]*len(transitive_train), test_size=0.50, random_state=36)
len(X_train)

In [ ]:
def last_index(list_of_elems, elem):
  return len(list_of_elems) - list_of_elems[::-1].index(elem) - 1

def extract_ne(sentence):
  output = predictor_ner.predict(sentence)
  tags = output['tags']
  words = output['words']
  span1_found = False
  entity1_idx = []
  entity2_idx = []
  for index, tag in enumerate(tags):
    if tag[0] == 'B':
      if not span1_found:
        entity1_idx.append(index)
      else:
        entity2_idx.append(index)
    elif tag[0] == 'L':
      if not span1_found:
        entity1_idx.append(index+1)
      else:
        entity2_idx.append(index+1)
      span1_found = True
    elif tag[0] == 'U':
      if not span1_found:
        entity1_idx.append(index)
        entity1_idx.append(index+1)
      else:
        entity2_idx.append(index)
        entity2_idx.append(index+1)
      span1_found = True

  entity1 = words[entity1_idx[0]:entity1_idx[1]]
  entity2 = words[entity2_idx[0]:entity2_idx[1]]
  return entity1, entity2

def extract_ner_span(sentence, e1, e2):
  tokens = sentence.split()
  span1 = [tokens.index(e1[0]), tokens.index(e1[-1])+1]
  span2 = [tokens.index(e2[0]), tokens.index(e2[-1])+1]
  assert ' '.join(tokens[span1[0]:span1[1]]) == ' '.join(e1)
  assert ' '.join(tokens[span2[0]:span2[1]]) == ' '.join(e2)
  return span1, span2

def build_example_kg(sentence, hypothesis, label, index):
  text = sentence.replace(": ", "").replace("? ", "").replace(" ,", "")
  e1, e2 = extract_ne(hypothesis)
  span1, span2 = extract_ner_span(text, e1, e2)
  example = {"text": text,
             "info": {"id": index},
             "targets": [
               {"label": str(label), 
                "span1": span1, "span2": span2, 
                "info": {"comment": ""}}]}
  return example

#entity1, entity2 = extract_ne("David Macaulay wrote The Way Things Work .")
#extract_ner_span("In college and after , I became a fan of David Macaulay , whose books on technology include The Way Things Work .", entity1, entity2)
#entity1, entity2 = extract_ne("Germany had a Nazi government .")
#extract_ner_span("Norman Baillie-Stewart , who was charged with passing information to the Nazi government in Germany .", entity1, entity2)
build_example_kg("In college and after , I became a fan of David Macaulay , whose books on technology include The Way Things Work .", "David Macaulay wrote The Way Things Work .", True, 0)

In [ ]:
transitive_train_data = []
exceptions_train = []
for index, item in tqdm(enumerate(X_train)):
  sentence = item['context']
  hypothesis = item['hypothesis']
  label = item['binary-label']
  try:
    example = build_example_kg(sentence, hypothesis, label, index)
    transitive_train_data.append(example)
  except:
    exceptions_train.append((index, sentence))
    continue

In [ ]:
transitive_val_data = []
exceptions_val = []
for index, item in tqdm(enumerate(transitive_val)):
  sentence = item['context']
  hypothesis = item['hypothesis']
  label = item['binary-label']
  try:
    example = build_example_kg(sentence, hypothesis, label, index)
    transitive_val_data.append(example)
  except:
    print(sentence)
    exceptions_val.append((index, sentence))
    #traceback.print_exc()

## Creating sample Edge-Probing data.

Because the Edge-Probing data is not publicly available, we will simulate the run with a single example. We will write 1000 copies for the training set and 100 copies for the validation set. We will also write the corresponding task config.

In [3]:
import jiant.utils.python.io as py_io

#train_data_raw = py_io.read_jsonl("/content/tasks/data/relational/train.jsonl", mode="r")
val_data_raw = py_io.read_json("./curriculum/socialQA.json", mode="r")
#for key in relation_data.keys():
#  train_data.append(relation_data[key])
#for key in relation_data_val.keys():
#  val_data.append(relation_data_val[key])

In [4]:
val_data = []
for example in val_data_raw:
  label = "entailment"
  if example['label'] == "neutral":
    label = example['label']
  val = {
    'sentence1': example['premise'],
    'sentence2': example['hypothesis'],
    'gold_label': label
  }
  val_data.append(val)

In [ ]:
set([target['label'] for target in train_data[7]['targets']])

In [ ]:
train_data[1]

In [ ]:
CONTEXT_TOKEN = "<S>"
SPAN1_TOKEN = "<SP1>"
SPAN2_TOKEN = "<SP2>"
MASK_TOKEN = "[MASK]"

def generate_prompt(example, prompts):
  template = "<S> Based on the given context , <SP1> ? [MASK] <SP2>"
  for target in example['targets']:
    prompt = template.replace(CONTEXT_TOKEN, example['text'])
    sequence = example['text'].split(" ")
    span1_idx = target['span1']
    span2_idx = target['span2']
    label = target['label']
    span1 = sequence[int(span1_idx[0]):int(span1_idx[1])]
    span2 = sequence[int(span2_idx[0]):int(span2_idx[1])]
    prompt = prompt.replace(SPAN1_TOKEN, " ".join(span1))
    prompt = prompt.replace(SPAN2_TOKEN, " ".join(span2))
    mask_idx = prompt.split(" ").index(MASK_TOKEN)
    new_example = {
      'text': prompt,
      'masked_spans': [[mask_idx, mask_idx+1]],
      'target': label
    }
    prompts.append(new_example)

In [ ]:
prompts_train = []
prompts_val = []

for data in train_data:
  generate_prompt(data, prompts_train)
for data_val in train_data:
  generate_prompt(data_val, prompts_val)

In [ ]:
print(prompts_train[0])
prompts_train[0]['text'].split(" ")[49:50]

In [5]:
task_name = "socialqa_nli"

os.makedirs("/content/tasks/configs/", exist_ok=True)
os.makedirs(f"/content/tasks/data/{task_name}", exist_ok=True)
py_io.write_jsonl(
    data=val_data, #train_darta,
    path=f"/content/tasks/data/{task_name}/train.jsonl",
)
py_io.write_jsonl(
    data=val_data,
    path=f"/content/tasks/data/{task_name}/val.jsonl",
)
py_io.write_json({
  "task": f"{task_name}",
  "paths": {
    "train": f"/content/tasks/data/{task_name}/train.jsonl",
    "val": f"/content/tasks/data/{task_name}/val.jsonl",
  },
  "name": f"{task_name}"
}, f"/content/tasks/configs/{task_name}_config.json")

#### Download model

Next, we will download a `roberta-base` model. This also includes the tokenizer.

In [ ]:
export_model.export_model(
    hf_pretrained_model_name_or_path="bert-base-uncased",
    output_base_path="./models/bert-base-uncased",
)

#### Tokenize and cache

With the model and data ready, we can now tokenize and cache the inputs features for our task. This converts the input examples to tokenized features ready to be consumed by the model, and saved them to disk in chunks.

In [17]:
import jiant.proj.main.tokenize_and_cache as tokenize_and_cache

bert1 = "bert-base-uncased"
bert2 = "bert-large-uncased"
roberta1 = "roberta-base"
roberta2 = "roberta-large"
deberta = "microsoft/deberta-base"

task_name = "lexical_nli"
model_name = bert1

tokenize_and_cache.main(tokenize_and_cache.RunConfiguration(
    task_config_path=f"/content/tasks/configs/{task_name}_config.json",
    hf_pretrained_model_name_or_path=model_name,
    output_dir=f"./cache/{model_name}/{task_name}",
    phases=["train", "val"],
))

LexicalNLITask
  [train]: /content/tasks/data/lexical_nli/train.jsonl
  [val]: /content/tasks/data/lexical_nli/val.jsonl


Tokenizing: 100%|██████████| 8193/8193 [00:03<00:00, 2313.73it/s]


In [25]:
metrics = py_io.read_json("./runs/socialqa_nli/roberta-base/overall_metrics.json")
acc_all = 0
for metric in metrics:
  acc_all += metric['aggregated']
acc_all / 10

0.36200233216724975

In [24]:
metrics = py_io.read_json("./runs/lexical_nli/roberta-base/overall_metrics.json")
acc_all = 0
for metric in metrics:
  acc_all += metric['aggregated']
acc_all / 10

0.3239594776028317

We can inspect the first examples of the first chunk of each task.

In [ ]:
import jiant.shared.caching as caching
row = caching.ChunkedFilesDataCache("./cache/bert-large-uncased/semgraph2/val").load_chunk(0)[10]["data_row"]
print(row.input_ids)
print(row.label_ids)
print(row.tokens)
print(row.tokens[row.spans[0][0]: row.spans[0][1]+1])
print(row.tokens[row.spans[1][0]: row.spans[1][1]+1])

In [5]:
import torch
model = torch.load("./models/microsoft/deberta-base-mnli/model/model.p")
model.keys()

odict_keys(['deberta.embeddings.position_ids', 'deberta.embeddings.word_embeddings.weight', 'deberta.embeddings.LayerNorm.weight', 'deberta.embeddings.LayerNorm.bias', 'deberta.encoder.layer.0.attention.self.q_bias', 'deberta.encoder.layer.0.attention.self.v_bias', 'deberta.encoder.layer.0.attention.self.in_proj.weight', 'deberta.encoder.layer.0.attention.self.pos_proj.weight', 'deberta.encoder.layer.0.attention.self.pos_q_proj.weight', 'deberta.encoder.layer.0.attention.self.pos_q_proj.bias', 'deberta.encoder.layer.0.attention.output.dense.weight', 'deberta.encoder.layer.0.attention.output.dense.bias', 'deberta.encoder.layer.0.attention.output.LayerNorm.weight', 'deberta.encoder.layer.0.attention.output.LayerNorm.bias', 'deberta.encoder.layer.0.intermediate.dense.weight', 'deberta.encoder.layer.0.intermediate.dense.bias', 'deberta.encoder.layer.0.output.dense.weight', 'deberta.encoder.layer.0.output.dense.bias', 'deberta.encoder.layer.0.output.LayerNorm.weight', 'deberta.encoder.layer

#### Writing a run config

Here we are going to write what we call a `jiant_task_container_config`. This configuration file basically defines a lot of the subtleties of our training pipeline, such as what tasks we will train on, do evaluation on, batch size for each task. The new version of `jiant` leans heavily toward explicitly specifying everything, for the purpose of inspectability and leaving minimal surprises for the user, even as the cost of being more verbose.

We use a helper "Configurator" to write out a `jiant_task_container_config`, since most of our setup is pretty standard. 

**Depending on what GPU your Colab session is assigned to, you may need to lower the train batch size.**

In [ ]:
def train_configuration(task_name, model_name):
    jiant_run_config = configurator.SimpleAPIMultiTaskConfigurator(
        task_config_base_path="/content/tasks/configs/",
        task_cache_base_path=f"./cache/{model_name}/",
        train_task_name_list=[task_name],
        val_task_name_list=[task_name],
        train_batch_size=8,
        eval_batch_size=16,
        epochs=5,
        num_gpus=1,
    ).create_config()

    os.makedirs("./run_configs/", exist_ok=True)
    py_io.write_json(jiant_run_config,
                     f"./run_configs/{task_name}_run_config.json")
    display.show_json(jiant_run_config)

def train(task_name, model_name):
    run_args = main_runscript.RunConfiguration(
        jiant_task_container_config_path=f"./run_configs/{task_name}_run_config.json",
        output_dir=f"./runs/{task_name}",
        hf_pretrained_model_name_or_path=model_name,
        model_path=f"./models/{model_name}/model/model.p",
        model_config_path=f"./models/{model_name}/model/config.json",
        learning_rate=1e-5,
        eval_every_steps=500,
        do_train=True,
        do_val=True,
        do_save=True,
        write_val_preds=True,
        freeze_encoder=True,
        force_overwrite=True,
        no_cuda=False
    )
    main_runscript.run_loop(run_args)

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
import torch

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')

inputs = tokenizer("There are <mask> people walking, a man, a woman, and two childern.", return_tensors="pt")
labels = tokenizer("There are 4 people walking, a man, a woman, and two childern.", return_tensors="pt")["input_ids"]

outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits

In [ ]:
input = tokenizer.encode("A person who diagnose illnesses works at the hospital. The hospital is where a doctor works. Based on the context, the word illnesses is a <mask>", return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]
token_logits = model(input).logits
mask_token_logits = token_logits[0, mask_token_index, :]
top_3_tokens = torch.topk(mask_token_logits, 3, dim=1).indices[0].tolist()
for token in top_3_tokens:
  print(tokenizer.decode([token]))

In [ ]:
doctor_def = ["helps a sick person", "treats a patient", "cures a person's disease",
              "diagnoses illnesses", "prescribes medicine"]
lawyer_def = ['represents you in court', 'objects in court', 'questions a witness',
              'rests the case', 'settles a law suit']
teacher_def = ['teaches students', 'assigns homework', 'works in a school',
               'dares you to learn', 'explains a fact']
engineer_def = ['invents and designs structures', 'analyzes, builds, and tests complex systems'
                'forms the link between scientific discoveries to human and business needs',
                'develops new technological solutions',
                'applies engineering analysis in testing, production, or maintenance']
entrepreneur_def = ['creates a new business', 'takes on the risks of creating a startup',
                    'organizes and operates a business', 'needs to ensure funding',
                    'brings good new ideas to market']
firefighter_def = ['is resbonsible for fire and rescure',
                   'rushes to a burning building', 'pulls a hoes',
                   'rescues people from dangerous situations',
                   'carries out a ladder slider', 'extinguish fires']
sailor_def = ['captains a ship', 'sails a ship', 'ties a knot',
              'sounds the depth of the sea', 'boards a boat']
actor_def = ['stars in a movie or TV show', 'acts scene in a play',
             'puts on a costume', 'appears at the theater', 'pictures a scene']
architect_def = ['positions walls and plumbing with exactness', 'completes a floor plan',
                 'plans, designs and oversees the construction of buildings',
                 'pictures a structural design',
                 'illustrates and generates bulding design proposals']
nurse_def = ['cares for a patient', 'pages the doctor',
             'checks vital signs', 'hands the instrument to the doctor', 'takes a pulse']
journalist_def = ['reads a press release', 'reports news', 'covers a story',
                  'interviews strangers with a microphone', 'notes a discrepancy in the facts']
policeman_def = ["is in a police force", 'prevents and detects crimes',
                 'protects and assists the general public', 'arrests criminals', 'investigates a crime']
writer_def = ['write a novel', 'words things for impact', 'writes poems',
              'words a phrase carefully', 'tells a story in a book']
programmer_def = ['step through a computer program', 'writes a software',
                  'sets up a website', 'understands programming language', 'fix a software bug']
carpenter_def = ['builds a house', 'hammers a nail', 'plains a door',
                 'nail furniture together', 'build roof, cabninets, and shelves']
baker_def = ['rolls dough', 'makes a cake', 'bake a wedding cake', 'flower a table',
             'coat a cake with lemon-flavored frosting']
chef_def = ['prepares food', 'works at restaurants and hotels', 'cooks a gourmet meal',
            'cooks very well', 'can bread chicken']
accountant_def = ['is a practitioner of accounting',
                  "has the ability to certify an organization's financial statements",
                  'complete a tax return', 'audits books for different companies', 'cooks the books']
farmer_def = ['farms the land', 'seeds the field', 'waters and gather the crops',
              'normaly owns a barn', 'feeds many farm animals']
scientist_def = ['conducts scientific research to advance knowledge',
                 'has an advanced degrees in an area of science',
                 'exhibits a strong curiosity about reality',
                 'applies scientific knowledge for the benefit of people',
                 'performs experiments to test hypotheses']

In [ ]:
computer_def = []
pencil_def = []
watch = []
phone = []
baterry = []
car = []
drills = []
dictionary_def = ['A tool that people use to look up word definition']
soap = []
lamp = []
mirror = []
cooker = []


In [ ]:
airport_def = ['is where people get on airplanes',
               'has an air traffic control tower',
               'is used for airplane landing and taking off',
               '',
               '']
hotel_def = ['is a place to stay when people travel',
             'is where people can book a room',
             'is used for a bed away from home',
             'is an establishment providing accommodations for travelers',
             'can provide paid lodging on a short-term basis']
bank_def = ['is a financial institution that accepts deposits from the public',
            'can create a demand deposit',
            'plays an important role in financial stability and the economy of a country',
            'is a subject to minimum capital requirements',
            'is a place where people can apply for credit cards']
hospital_def = ['is used for healing sick people',
                'is a place to have surgery',
                'is for treating seriously injured people',
                'is a workplace for doctors and nurses',
                'is able to handle medical emergencies']
school_def = ['is a place for learning and teaching the students',
              'is where people can get an education',
              'provides learning spaces and learning environments',
              'is where teachers work',
              'typically has classrooms, a library, and a cafeteria']
church_def = ['is where people pray to God']
court_def = []
cinema_def = []
gym_def = []
farm_def = []
gallery_def = []

In [ ]:
mountain_def = []

In [ ]:
input = tokenizer.encode("A tool that people use to look up word definition is a <mask>", return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]
token_logits = model(input).logits
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
for token in top_5_tokens:
  print(tokenizer.decode([token]))

In [ ]:
input = tokenizer.encode("Lava and volcanic ash comes from <mask>", return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]
token_logits = model(input).logits
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
for token in top_5_tokens:
  print(tokenizer.decode([token]))

In [ ]:
input = tokenizer.encode("A professional who applies engineering analysis in testing, production, or maintenance is an <mask>", return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]
token_logits = model(input).logits
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
for token in top_5_tokens:
  print(tokenizer.decode([token]))

In [ ]:

input = tokenizer.encode("A person who performs experiments to test hypotheses is a <mask>", return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]
token_logits = model(input).logits
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
for token in top_5_tokens:
  print(tokenizer.decode([token]))

In [ ]:
input = tokenizer.encode("A man, a woman, and two children are walking on the beach. <mask> people are walking on the beach.", return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]
token_logits = model(input).logits
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
for token in top_5_tokens:
  print(tokenizer.decode([token]))

In [ ]:
input = tokenizer.encode("Bert ate 2 <mask>, cake, ice cream, mochi, cotton candy, spinach, and cabbage.", return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]
token_logits = model(input).logits
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
for token in top_5_tokens:
  print(tokenizer.decode([token]))

In [ ]:
import torch

bart = torch.hub.load('pytorch/fairseq', 'bart.base')
bart.cuda()
bart.eval()
bart.fill_mask(['The cat <mask> on the <mask>.'], topk=3, beam=10, match_source_len=False)

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

ARTICLE_TO_SUMMARIZE = "While at Skidmore , Smith also designed an even taller mixed-use skyscraper , the Burj Dubai , now under construction in the United Arab Emirates ."
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')

# Generate Summary
summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=5, early_stopping=True)
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

In [ ]:
ARTICLE_TO_SUMMARIZE = "While at Skidmore , Smith also designed an even taller mixed-use skyscraper , the Burj Dubai , now under construction in the United Arab Emirates ."
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')

# Generate Summary
summary_ids = model.generate(inputs['input_ids'], num_beams=5, max_length=10, early_stopping=True)
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

Since we're training and evaluating on the same (duplicated) example, we should get perfect performance, but hopefully this notebook should be illustrative of the workflow for edge-probing tasks.